# MAPPO - IPPO algorithms implementation

> In this notebook, we implement two state-of-the-art Multi Agent Reinforcement Leaning (MARL) algorithms **Multi-Agent Proximal Policy Optimization [MAPPO](https://arxiv.org/pdf/2103.01955)** and **Independent Proximal Policy Optimization [IPPO](https://arxiv.org/pdf/2011.09533)** in our environment. 


> Tutorial based on [Multi-Agent Reinforcement Learning (PPO) with TorchRL Tutorial](https://pytorch.org/rl/stable/tutorials/multiagent_ppo.html).

### Simulation overview

> We simulate our environment with an initial population of **100 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **20 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ either the MAPPO or IPPO reinforcement learning algorithms to further refine their learning.

![Alt text](../../docs/img/env.png)


#### Imported libraries

In [1]:
%run startup.py

In [2]:
import os
import sys
import torch
from tqdm import tqdm

from tensordict.nn import TensorDictModule
from torchrl.collectors import SyncDataCollector
from torch.distributions import Categorical
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import MultiAgentMLP, ProbabilisticActor
from torchrl.objectives.value import GAE
from torchrl.objectives import ClipPPOLoss, ValueEstimators

from routerl import TrafficEnvironment

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#### Hyperparameters setting

In [3]:
# Devices
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

print("device is: ", device)
vmas_device = device  # The device where the simulator is run

# Sampling
frames_per_batch = 100  # Number of team frames collected per training iteration
n_iters = 50  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 1  # Number of optimization steps per training iteration
minibatch_size = 16  # Size of the mini-batches in each optimization step
lr = 3e-43 # Learning rate
max_grad_norm = 3.0  # Maximum norm for the gradients

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.99  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss


policy_network_depth=3
policy_network_num_cells = 64

critic_network_depth=3
critic_network_num_cells = 64

# Human learning phase
human_learning_episodes = 100

env_params = {
    "agent_parameters" : {
        "num_agents" : 100,
        "new_machines_after_mutation": 20,
        "human_parameters" : {
            "model" : "w_avg"
        },
    },
    "simulator_parameters" : {
        "network_name" : "ingolstadt"
    },  
    "plotter_parameters" : {
        "phases" : [0, human_learning_episodes],
        "smooth_by" : 50,
    }
}

device is:  cpu


#### Environment initialization

> In this example, the environment initially contains only human agents.

In [4]:
env = TrafficEnvironment(seed=42, **env_params)

print(env)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: C:\Program Files (x86)\Eclipse\Sumo\tools
TrafficEnvironment with 100 agents.            
0 machines and 100 humans.            
Machines: []            
Humans: [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19, Human 20, Human 21, Human 22, Human 23, Human 24, Human 25, Human 26, Human 27, Human 28, Human 29, Human 30, Human 31, Human 32, Human 33, Human 34, Human 35, Human 36, Human 37, Human 38, Human 39, Human 40, Human 41, Human 42, Human 43, Human 44, Human 45, Human 46, Human 47, Human 48, Human 49, Human 50, Human 51, Human 52, Human 53, Human 54, Human 55, Human 56, Human 57, Human 58, Human 59, Human 60, Human 61, Human 62, Human 63, Human 64, Human 65, Human 66, Human 67, Human 68, Human 69, Human 70, Human 71, Human 72, Human 73, Human 74, Human 75, Hu

In [5]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  100 

Agents are:  [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19, Human 20, Human 21, Human 22, Human 23, Human 24, Human 25, Human 26, Human 27, Human 28, Human 29, Human 30, Human 31, Human 32, Human 33, Human 34, Human 35, Human 36, Human 37, Human 38, Human 39, Human 40, Human 41, Human 42, Human 43, Human 44, Human 45, Human 46, Human 47, Human 48, Human 49, Human 50, Human 51, Human 52, Human 53, Human 54, Human 55, Human 56, Human 57, Human 58, Human 59, Human 60, Human 61, Human 62, Human 63, Human 64, Human 65, Human 66, Human 67, Human 68, Human 69, Human 70, Human 71, Human 72, Human 73, Human 74, Human 75, Human 76, Human 77, Human 78, Human 79, Human 80, Human 81, Human 82, Human 83, Human 84, Human 85, Human 86, Human 87, Human 88, Human 89, Human 90, Human 91, Human 92, Human 93, Human 94, Human 95, Hu

> Reset the environment and the connection with SUMO

In [6]:
env.start()
env.reset()

({}, {})

#### Human learning

In [7]:
for episode in range(human_learning_episodes):
    env.step()

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles). You can adjust the number of agents to be mutated in the <code style="color:white">/params.json</code> file.

In [8]:
env.mutation()

In [9]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  100 

Agents are:  [Machine 1, Machine 15, Machine 10, Machine 91, Machine 22, Machine 73, Machine 5, Machine 52, Machine 81, Machine 77, Machine 23, Machine 62, Machine 20, Machine 7, Machine 47, Machine 56, Machine 8, Machine 57, Machine 35, Machine 41, Human 0, Human 2, Human 3, Human 4, Human 6, Human 9, Human 11, Human 12, Human 13, Human 14, Human 16, Human 17, Human 18, Human 19, Human 21, Human 24, Human 25, Human 26, Human 27, Human 28, Human 29, Human 30, Human 31, Human 32, Human 33, Human 34, Human 36, Human 37, Human 38, Human 39, Human 40, Human 42, Human 43, Human 44, Human 45, Human 46, Human 48, Human 49, Human 50, Human 51, Human 53, Human 54, Human 55, Human 58, Human 59, Human 60, Human 61, Human 63, Human 64, Human 65, Human 66, Human 67, Human 68, Human 69, Human 70, Human 71, Human 72, Human 74, Human 75, Human 76, Human 78, Human 79, Human 80, Human 82, Human 83, Human 84, Human 85, Human 86, Human 87, Human 88, Human 89, Human 90, Hu

#### PettingZoo environment wrapper

In [10]:
group = {'agents': [str(machine.id) for machine in env.machine_agents]}

env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

#### Transforms

Here we instantiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [11]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [12]:
check_env_specs(env)


2025-02-05 13:46:03,382 [torchrl][INFO] check_env_specs succeeded!


In [13]:
reset_td = env.reset()

#### Policy/Actor network

In [14]:
share_parameters_policy = False 

policy_net = torch.nn.Sequential(
    MultiAgentMLP(
        n_agent_inputs = env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs = env.action_spec.space.n,
        n_agents = env.n_agents,
        centralised=False,
        share_params=share_parameters_policy,
        device=device,
        depth=policy_network_depth,
        num_cells=policy_network_num_cells,
        activation_class=torch.nn.Tanh,
    ),
)

In [15]:
policy_module = TensorDictModule(
    policy_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "logits")],
) 

In [16]:
policy = ProbabilisticActor(
    module=policy_module,
    spec=env.action_spec,
    in_keys=[("agents", "logits")],
    out_keys=[env.action_key],
    distribution_class=Categorical,
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)

#### Critic network

> The critic reads the observations and returns the corresponding value estimates.

In [17]:
share_parameters_critic = True
mappo = False  # IPPO if False

critic_net = MultiAgentMLP(
    n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
    n_agent_outputs=1, 
    n_agents=env.n_agents,
    centralised=mappo,
    share_params=share_parameters_critic,
    device=device,
    depth=critic_network_depth,
    num_cells=critic_network_num_cells,
    activation_class=torch.nn.ReLU,
)

critic = TensorDictModule(
    module=critic_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "state_value")],
)

In [18]:
print("Running policy:", policy(env.reset()))

Running policy: TensorDict(
    fields={
        agents: TensorDict(
            fields={
                action: Tensor(shape=torch.Size([20]), device=cpu, dtype=torch.int64, is_shared=False),
                done: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                logits: Tensor(shape=torch.Size([20, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([20]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([20, 4]), device=cpu, dtype=torch.float32, is_shared=False),
                sample_log_prob: Tensor(shape=torch.Size([20]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(sh

In [19]:
print("Running value:", critic(env.reset()))

Running value: TensorDict(
    fields={
        agents: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([20]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([20, 4]), device=cpu, dtype=torch.float32, is_shared=False),
                state_value: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([20, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([20]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([1]), de

#### Collector

In [20]:
collector = SyncDataCollector(
    env,
    policy,
    device=device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
) 

#### Replay buffer

In [21]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(
        frames_per_batch, device=device
    ),  
    sampler=SamplerWithoutReplacement(),
    batch_size=minibatch_size,
)

#### PPO loss function

In [22]:
loss_module = ClipPPOLoss(
    actor_network=policy,
    critic_network=critic,
    clip_epsilon=clip_epsilon,
    entropy_coef=entropy_eps,
    normalize_advantage=False,
)
loss_module.set_keys( 
    reward=env.reward_key,  
    action=env.action_key, 
    sample_log_prob=("agents", "sample_log_prob"),
    value=("agents", "state_value"),
    done=("agents", "done"),
    terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(
    ValueEstimators.GAE, gamma=gamma, lmbda=lmbda
) 

GAE = loss_module.value_estimator

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [23]:
pbar = tqdm(total=n_iters, desc="episode_reward_mean = 0")

episode_reward_mean_list = []
loss_values = []
loss_entropy = []
loss_objective = []
loss_critic = []

for tensordict_data in collector: ##loops over frame_per_batch

    ## Generate the rollouts
    tensordict_data.set(
        ("next", "agents", "done"),
        tensordict_data.get(("next", "done"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )
    tensordict_data.set(
        ("next", "agents", "terminated"),
        tensordict_data.get(("next", "terminated"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )

    # Compute GAE for all agents
    with torch.no_grad():
            GAE(
                tensordict_data,
                params=loss_module.critic_network_params,
                target_params=loss_module.target_critic_network_params,
            )

    data_view = tensordict_data.reshape(-1)  
    replay_buffer.extend(data_view)

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)

            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            loss_value.backward()

            torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), max_grad_norm
            ) 

            optim.step()
            optim.zero_grad()

            loss_values.append(loss_value.item())

            loss_entropy.append(loss_vals["loss_entropy"].item())

            loss_objective.append(loss_vals["loss_objective"].item())

            loss_critic.append(loss_vals["loss_critic"].item())


   
    collector.update_policy_weights_()
   
    # Logging
    done = tensordict_data.get(("next", "agents", "done"))  # Get done status for the group

    episode_reward_mean = (
        tensordict_data.get(("next", "agents", "episode_reward"))[done].mean().item()
    )
    episode_reward_mean_list.append(episode_reward_mean)


    pbar.set_description(f"episode_reward_mean = {episode_reward_mean}", refresh=False)
    pbar.update()

episode_reward_mean = -11.279000282287598: 100%|██████████| 50/50 [11:56<00:00, 13.08s/it]

>  Check `\plots` directory to find the plots created from this experiment.

In [24]:
env.plot_results()